In [1]:
import numpy as np
import random
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
#train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
#query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
#labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
#gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
#filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
#camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:
#scaler = StandardScaler()
print(np.array(features))
#features = scaler.fit_transform(features)
X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)

[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]
[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]


In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]

In [11]:
#len(X_train)

7368

In [12]:
#len(set(y_train))

767

In [13]:
#len(X_query)

1400

In [14]:
#len(X_gallery)

5328

In [15]:
#len(y_train)

7368

In [16]:
import pandas as pd 
import numpy as np

from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.models import Sequential

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K

from sklearn import preprocessing, model_selection 
from keras.wrappers.scikit_learn import KerasRegressor

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
# create model
model = Sequential()
# layer 1
model.add(Dense(6144, input_dim=6144, activation='relu', kernel_initializer='normal'))
#layer 2
model.add(Dense(1024, activation='relu', kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#layer 3
model.add(Dense(128, activation='relu',kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#layer 4
model.add(Dense(2, activation='softmax'))
# Compile model
from keras import metrics
#optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov= False)
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer, loss='categorical_crossentropy') 

In [18]:
X_train_pairs = []
y_train_pair_lables = []
for Xtnn, ytnn, camId_t in zip(X_train, y_train, camId_train):
    for Xtnn2, ytnn2, camId_t2 in zip(X_train, y_train, camId_train):
        if ((camId_t == camId_t2) and (ytnn == ytnn2)):
            continue
        elif (ytnn == ytnn2):
            #for i in range (30):
            randindex = random.randint(0, 6401)
            Xtnn3 = X_train[randindex]
            ytnn3 = y_train[randindex]                 

            dist2 = 0
            if(ytnn == ytnn3):
                dist3 = 0
            else:
                dist3 = 1

            Xconcat = np.concatenate((Xtnn,Xtnn2,Xtnn3), axis = None)
            X_train_pairs.append(Xconcat)
            y_train_pair_lables.append((dist2,dist3))   

            Xconcat = np.concatenate((Xtnn,Xtnn3,Xtnn2), axis = None)
            X_train_pairs.append(Xconcat)
            y_train_pair_lables.append((dist3,dist2)) 

y_train_pair_lables = np.array(y_train_pair_lables)
X_train_pairs = np.array(X_train_pairs)
model.fit(X_train_pairs, y_train_pair_lables, batch_size = 150, epochs=20)

Epoch 1/20
70784/70784 [==============================] - 300s 4ms/step - loss: 0.1076
Epoch 2/20
70784/70784 [==============================] - 265s 4ms/step - loss: 0.0117
Epoch 3/20
70784/70784 [==============================] - 267s 4ms/step - loss: 0.0090
Epoch 4/20
70784/70784 [==============================] - 267s 4ms/step - loss: 0.0082
Epoch 5/20
70784/70784 [==============================] - 265s 4ms/step - loss: 0.0047
Epoch 6/20
70784/70784 [==============================] - 268s 4ms/step - loss: 0.0047
Epoch 7/20
70784/70784 [==============================] - 268s 4ms/step - loss: 0.0057
Epoch 8/20
70784/70784 [==============================] - 269s 4ms/step - loss: 0.0038
Epoch 9/20
70784/70784 [==============================] - 270s 4ms/step - loss: 0.0048
Epoch 10/20
70784/70784 [==============================] - 264s 4ms/step - loss: 0.0040
Epoch 11/20
70784/70784 [==============================] - 265s 4ms/step - loss: 0.0042
Epoch 12/20
70784/70784 [================

In [19]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [ ]:
rank_accuracies = []
AP = []
for val, camId_v, y_v  in zip(X_query, camId_query, y_query):
    v_v_dists = []
    y_valid = []
    for val2, camId_v2, y_v2  in zip(X_gallery, camId_gallery, y_gallery):
        if ((camId_v == camId_v2) and (y_v == y_v2)):
            continue
        else:
            randindex = random.randint(0, 772)
            val3 = X_gallery[randindex]
            y_v3 = y_gallery[randindex]

            dist = model.predict(np.concatenate((val, val2, val3)).reshape((1,6144)))[0][0]
            v_v_dists.append(dist)
            y_valid.append(y_v2)

    tot_label_occur = y_valid.count(y_v)
    v_v_dists = np.array(v_v_dists)
    y_valid = np.array(y_valid)

    
    _indexes = np.argsort(v_v_dists)

    # Sorted distances and labels
    v_v_dists, y_valid = v_v_dists[_indexes], y_valid[_indexes]
    print(v_v_dists)
    
    if tot_label_occur != 0:
        AP_, rank_A = get_acc_score(y_valid, y_v, tot_label_occur)

        AP.append(AP_)

        rank_accuracies.append(rank_A) 

    #if q  > 5:
    #    break
    #q = q+1

rank_accuracies = np.array(rank_accuracies)

total = rank_accuracies.shape[0]
rank_accuracies = rank_accuracies.sum(axis = 0)
rank_accuracies = np.divide(rank_accuracies, total)

i = 0
print ('Accuracies by Rank:')
while i < rank_accuracies.shape[0]:
    print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
          'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
          'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
          'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
          'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
    i = i+5

AP = np.array(AP)

mAP = AP.sum()/AP.shape[0]
print('mAP = %.2f%%' % (mAP * 100))

[5.0233727e-11 8.1137895e-11 9.5148345e-10 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[2.8700351e-09 3.2866978e-09 9.9557873e-09 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.2381496e-10 2.8080663e-10 1.1443680e-09 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[6.7580688e-09 2.1555156e-08 4.1491074e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[4.5438364e-10 2.0828789e-08 3.4788705e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.3292725e-11 5.3739051e-11 6.0368606e-11 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.0353378e-10 1.3309426e-10 1.4738374e-10 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[2.6903089e-12 1.3600013e-11 1.7607321e-10 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[4.6050459e-08 8.5439632e-07 1.1167148e-06 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[4.6135565e-08 1.2002999e-07 2.4387771e-07 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.4704302e-08 2.0861231e-08 2.9291895e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]

[2.2424493e-09 2.4951425e-09 1.4415000e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[9.4891303e-11 2.2999565e-10 4.4921927e-10 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[8.8688390e-09 2.3819275e-08 2.6397991e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.2660382e-09 1.6414216e-09 2.5870908e-09 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.0705398e-11 2.5232824e-10 3.2311520e-10 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[6.1732646e-09 1.1092617e-08 1.3184723e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[8.0140972e-10 1.7418272e-09 1.3012316e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.3639644e-09 5.7214806e-09 2.0670560e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[4.01799483e-08 6.14039806e-08 1.09640986e-07 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
[9.7838193e-10 9.5448671e-09 2.0330036e-08 ... 1.0000000e+00 1.0000000e+00
 1.0000000e+00]
[1.1771774e-09 1.2769880e-09 5.6753100e-09 ... 1.0000000e+00 1.0000000e+00
 1.000000